# Laboratorio no. 5 Threat Hunting
Marco Jurado 20308

In [40]:
import pandas as pd
import json

Abrimos large_eve.json y luego verificamos tener los elementos correctos

In [41]:
json_file_path = 'large_eve.json'
data = []

with open(json_file_path, 'r') as file:
    for line in file:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON en la línea: {e}")

print(f'Total de registros cargados: {len(data)}')

df = pd.DataFrame(data)
print(df.info())

Total de registros cargados: 746909
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746909 entries, 0 to 746908
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   timestamp     746909 non-null  object 
 1   flow_id       746892 non-null  float64
 2   pcap_cnt      189115 non-null  float64
 3   event_type    746909 non-null  object 
 4   vlan          188065 non-null  float64
 5   src_ip        746892 non-null  object 
 6   src_port      746278 non-null  float64
 7   dest_ip       746892 non-null  object 
 8   dest_port     746278 non-null  float64
 9   proto         746892 non-null  object 
 10  alert         2145 non-null    object 
 11  ssh           6451 non-null    object 
 12  dns           15749 non-null   object 
 13  tx_id         88186 non-null   float64
 14  http          170981 non-null  object 
 15  app_proto     186205 non-null  object 
 16  fileinfo      82787 non-null   object 
 17  tls         

Filtramos DNS y verificamos tener exito al tener los elementos.

In [42]:
dns_records = [record for record in data if record.get('event_type') == 'dns']
print(f'Total de registros DNS: {len(dns_records)}')
df_dns = pd.json_normalize(dns_records)
print(f'Shape del DataFrame: {df_dns.shape}')
print(df_dns.head())
df_dns_a_records = df_dns[df_dns['dns.rrtype'] == 'A']
print(df_dns_a_records.sample(2))

Total de registros DNS: 15749
Shape del DataFrame: (15749, 18)
                         timestamp           flow_id  pcap_cnt event_type  \
0  2017-07-22T17:33:16.661646-0500  1327836194150542     22269        dns   
1  2017-07-22T17:33:24.990320-0500  2022925111925872     54352        dns   
2  2017-07-22T17:33:27.379891-0500   578544790391795     54519        dns   
3  2017-07-22T17:33:27.380146-0500   578544790391795     54520        dns   
4  2017-07-22T17:33:27.380146-0500   578544790391795     54520        dns   

   vlan                                   src_ip  src_port  \
0   110  2001:0dbb:0c18:0011:0260:6eff:fe30:0863     59680   
1   110  2001:0dbb:0c18:0011:0260:6eff:fe30:0863     38051   
2   150                          192.168.205.170     31393   
3   150                            192.168.207.4        53   
4   150                            192.168.207.4        53   

                                   dest_ip  dest_port proto dns.type  dns.id  \
0  2001:0500:0001:000

Mostramos dos elementos cualquiera

In [43]:
print(json.dumps(dns_records[0], indent=2))
print(json.dumps(dns_records[1], indent=2))

{
  "timestamp": "2017-07-22T17:33:16.661646-0500",
  "flow_id": 1327836194150542,
  "pcap_cnt": 22269,
  "event_type": "dns",
  "vlan": 110,
  "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
  "src_port": 59680,
  "dest_ip": "2001:0500:0001:0000:0000:0000:803f:0235",
  "dest_port": 53,
  "proto": "UDP",
  "dns": {
    "type": "query",
    "id": 15529,
    "rrname": "api.wunderground.com",
    "rrtype": "A",
    "tx_id": 0
  }
}
{
  "timestamp": "2017-07-22T17:33:24.990320-0500",
  "flow_id": 2022925111925872,
  "pcap_cnt": 54352,
  "event_type": "dns",
  "vlan": 110,
  "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
  "src_port": 38051,
  "dest_ip": "2001:0500:0003:0000:0000:0000:0000:0042",
  "dest_port": 53,
  "proto": "UDP",
  "dns": {
    "type": "query",
    "id": 58278,
    "rrname": "stork79.dropbox.com",
    "rrtype": "A",
    "tx_id": 0
  }
}


Mostramos la forma de DNS

In [44]:
print(df_dns.shape)

(15749, 18)


filtrar los registros DNS para aquellos registros tipo A

In [45]:
print(f"Total de registros tipo 'A': {len(df_dns_a_records)}")

Total de registros tipo 'A': 2849


Filtrar los dominios únicos

In [46]:
unique_a_records = df_dns_a_records['dns.rrname'].unique()
print(f"Total de dominios únicos tipo 'A': {len(unique_a_records)}")


Total de dominios únicos tipo 'A': 177


Función que extraiga el TLD de un dominio completo

In [47]:
from urllib.parse import urlparse

def get_tld(domain):
    if domain:
        if not domain.startswith(('http://', 'https://')):
            domain = 'http://' + domain
        tld = urlparse(domain).hostname.split('.')[-2:]
        return '.'.join(tld) if len(tld) == 2 else domain
    return ""

print(get_tld("api.wunderground.com"))
print(get_tld("safebrowsing.clients.google.com.home"))


wunderground.com
com.home


In [48]:
df_unique_a = pd.DataFrame(unique_a_records, columns=['domain'])

df_unique_a['domain_tld'] = df_unique_a['domain'].apply(get_tld)

df_unique_a = df_unique_a.drop(columns=['domain'])

print(df_unique_a)

           domain_tld
0    wunderground.com
1         dropbox.com
2           aoltw.net
3            com.home
4         mozilla.com
..                ...
172   com.localdomain
173     microsoft.com
174      verisign.com
175            22.201
176          real.com

[177 rows x 1 columns]
